# IDRNN
[Description]
(Add description here)

The goal of the project:
1. palceholder
2. placeholder


# Dataset

First download the dataset.

# Packages

In [ ]:
# !pip3 install -r requirements.txt

In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import torch
from tqdm import tqdm
from torch.utils.data import  Dataset, DataLoader
import torch.nn.functional as F



# load data and print it

In [ ]:
# from dataloader import ProteinDataset, proteinread
from dataloader import AminoAcidDataset

In [ ]:

input_csv_file = "data.csv"

max_sequence_length = 1000
num_amino_acids = 20

dataset = AminoAcidDataset(input_csv_file, max_sequence_length, num_amino_acids)

# Create a DataLoader
batch_size = 128
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)




In [ ]:
# pick up one sequence to show how our data looks like
g, g_len, target = dataset[65]
print(g)
print(g_len)
print(target)

In [ ]:

# Create an empty list to store the numerical sequences
training_sequences = []
# Iterate over the training_dataset
for sample in dataset:
    # Extract the numerical part (the first element of the tuple) and append it to the list
    training_sequences.append(sample[0])

# Print the list of training sequences
print(training_sequences[65])

In [ ]:
print(training_sequences)

# Helper function and calculation (kld reconstruct loss) and printer form tuorial 6

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

In [ ]:
## Help function from tutroial 6 to calculate loss and plotting

def count_parameters(model):
    total_params = sum(p.numel() for p in model.parameters())
    return total_params
def plot_losses(reco_losses, kld_losses):
    fig = plt.figure(figsize=(10, 5))

    ax = fig.add_subplot(1, 2, 1)
    ax.plot(reco_losses)
    ax.set_title('Reconstruction Loss')

    ax = fig.add_subplot(1, 2, 2)
    ax.plot(kld_losses)
    ax.set_title('KLD Loss')



# trainning loop based on tutorial 6

In [ ]:
from model_VAE import VAECNN

latent_dim = 512

vae = VAECNN(input_size=21, output_size=21, latent_dim=latent_dim)

if torch.cuda.is_available():
    vae.cuda()

In [ ]:
print(f'VAE has {count_parameters(vae):,} trainable parameters')

In [ ]:

optimizer = torch.optim.Adam(vae.parameters(), lr=1e-3)

beta = 0 # Weight of the KL divergence term
alpha = 1 # weight for cross entropy loss

def vae_loss(recon_x, target, mu, logvar):
    # Reconstruction loss
    recon_loss = []
    for batch in range(recon_x.shape[0]):
        recon_batch = recon_x[batch, :, :]
        target_batch = target[batch, :]
        # print(recon_batch.shape, target_batch.shape)
        cross_entropy_loss = F.cross_entropy(input=recon_batch.squeeze(0), target=target_batch.squeeze(0), reduction='mean')
        # print(cross_entropy_loss)
        recon_loss.append(cross_entropy_loss)
    recon_loss = torch.mean(torch.stack(recon_loss)) # stack the tensor to tensor list
    
    # Regularization term (KL divergence)
    kl_divergence = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    
    # Combine reconstruction loss and regularization term
    total_loss = (beta * kl_divergence) + (alpha * recon_loss)
    
    return kl_divergence, recon_loss, total_loss

num_epochs = 20
num_batches = len(dataloader)

total_losses = []
reco_losses = []
kld_losses  = []

for epoch in tqdm(range(num_epochs)):
    vae.train()
    temp_total_loss = 0
    temp_reco_loss = 0
    temp_kld_loss = 0
    
    
    for seq_batch, length_batch, target_batch in dataloader:
        if torch.cuda.is_available():
            seq_batch = seq_batch.cuda()
            length_batch = length_batch.cuda()
            target_batch = target_batch.cuda()
        optimizer.zero_grad()
        
        recon_batch, mean, logvar = vae(seq_batch)
        kld_loss, reco_loss, total_loss = vae_loss(recon_batch, target_batch, mean, logvar)
        
        temp_total_loss += total_loss
        temp_kld_loss += kld_loss
        temp_reco_loss += reco_loss
    
        total_loss.backward()
        optimizer.step()

    total_losses.append(total_loss.item())
    reco_losses.append(reco_loss.item())
    kld_losses.append(kld_loss.item())
    print(total_losses[epoch],  min(total_losses))
    if epoch == 0 or total_losses[epoch] <= min(total_losses):
        torch.save(vae.state_dict(), "vae_model.pt")
        print("Model saved")
        
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss:.4f}")


In [ ]:

plot_losses(reco_losses, kld_losses)